In [1]:
!pip install requests beautifulsoup4 lxml selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 16.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.2/161.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 11.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.4
    Uninstalling websocket-client-1.6.4:
      Successfully uninstalled websocket-client-1.6.4
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.13
    Uninstalling urllib3-1.26.13:
      Successfully uninstalled urllib3-1.26.13
  Attempting uninstall: typing_extensions
    Found existing installation: typi

In [5]:
# -*- coding: utf-8 -*-
"""
크롤러: https://www.cela.kr/ISMS_1_1
- 1) requests + BeautifulSoup로 정적 HTML 수집
- 2) 주요 콘텐츠(제목/본문/표/목록/링크) 구조화
- 3) 정적 파싱이 비거나 비정상일 경우 Selenium로 재시도(옵션)
- 4) 결과를 CSV/JSON으로 저장
"""

import os
import time
import json
import csv
import re
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional

import requests
from bs4 import BeautifulSoup

# ---- (옵션) 동적 렌더링 대비용: 필요 시 True ----
USE_SELENIUM_FALLBACK = True

# ---- 사용자 설정 ----
TARGET_URL = "https://www.cela.kr/ISMS_1_1"
OUT_DIR = "../data"
TIMEOUT = 20
RETRY = 3
SLEEP_SEC = 1.0

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/134.0.0.0 Safari/537.36"
    )
}

@dataclass
class ContentRow:
    section_order: int
    section_title: str
    content_type: str   # "paragraph" | "list" | "table" | "link"
    text: str           # 가공된 텍스트(표는 TSV 문자열)
    href: Optional[str] = None


def safe_mkdir(path: str):
    os.makedirs(path, exist_ok=True)


def fetch_html(url: str, headers: Dict[str, str], timeout: int = TIMEOUT) -> str:
    last_exc = None
    for _ in range(RETRY):
        try:
            resp = requests.get(url, headers=headers, timeout=timeout)
            if 200 <= resp.status_code < 300:
                return resp.text
        except Exception as e:
            last_exc = e
        time.sleep(0.8)
    if last_exc:
        raise last_exc
    raise RuntimeError("Failed to fetch HTML")


def load_with_selenium(url: str) -> str:
    """
    동적 렌더링이 필요한 경우를 대비한 백업 로더.
    - pip install selenium webdriver-manager
    - 로컬/서버에 Chrome 또는 Chromium 필요
    """
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--lang=ko-KR")
    for k, v in HEADERS.items():
        if k.lower() == "user-agent":
            options.add_argument(f"--user-agent={v}")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=options)
    try:
        driver.get(url)
        time.sleep(2.0)  # 간단 대기(필요 시 WebDriverWait으로 보강)
        return driver.page_source
    finally:
        driver.quit()


def extract_tables(soup: BeautifulSoup) -> List[str]:
    """
    표를 TSV 문자열로 변환하여 리스트 반환.
    """
    tables_tsv: List[str] = []
    for table in soup.select("table"):
        rows = []
        for tr in table.select("tr"):
            cells = tr.select("th, td")
            row = []
            for c in cells:
                txt = " ".join(c.get_text(" ", strip=True).split())
                row.append(txt)
            rows.append("\t".join(row))
        if rows:
            tables_tsv.append("\n".join(rows))
    return tables_tsv


def normalize_text(el) -> str:
    return " ".join(el.get_text(" ", strip=True).split())


def guess_sections(soup: BeautifulSoup) -> List[Dict]:
    """
    h1~h4를 섹션 헤더로 보고, 다음 헤더가 나오기 전까지의 형제 노드들을 그 섹션 내용으로 간주.
    페이지 구조가 다르면 필요한 부분만 골라 쓰면 됨.
    """
    headers = soup.select("h1, h2, h3, h4")
    sections = []
    for idx, h in enumerate(headers):
        title = normalize_text(h)
        content_nodes = []
        # 다음 헤더 전까지의 형제들을 수집
        for sib in h.find_all_next():
            if sib == h:
                continue
            if sib.name in ["h1", "h2", "h3", "h4"]:
                break
            # 너무 잡음이 많은 태그는 제외
            if sib.name in ["script", "style", "noscript"]:
                continue
            content_nodes.append(sib)
        sections.append({"title": title, "nodes": content_nodes})
    # 섹션 헤더가 전혀 없으면 body 기준으로 하나 생성
    if not sections:
        body = soup.body or soup
        sections = [{"title": "본문", "nodes": list(body.children)}]
    return sections


def parse_section(title: str, nodes: List) -> List[ContentRow]:
    rows: List[ContentRow] = []
    order = 0

    # 표
    temp_soup = BeautifulSoup("".join(str(n) for n in nodes), "html.parser")
    for tsv in extract_tables(temp_soup):
        rows.append(ContentRow(order, title, "table", tsv))
        order += 1

    # 문단
    for p in temp_soup.select("p"):
        txt = normalize_text(p)
        if txt:
            rows.append(ContentRow(order, title, "paragraph", txt))
            order += 1

    # 목록
    for lst in temp_soup.select("ul, ol"):
        items = []

        # 수정 후 (리스트 내부 모든 항목 수집)
        for li in lst.select("li"):
            t = normalize_text(li)
            if t:
                items.append(t)
        
        if items:
            rows.append(ContentRow(order, title, "list", "\n".join(items)))
            order += 1

    # 링크
    for a in temp_soup.select("a[href]"):
        href = a.get("href", "").strip()
        txt = normalize_text(a)
        if href and not href.startswith("javascript:"):
            rows.append(ContentRow(order, title, "link", txt, href=href))
            order += 1

    return rows


def crawl_cela_isms(url: str) -> List[ContentRow]:
    html = fetch_html(url, HEADERS)
    soup = BeautifulSoup(html, "html.parser")

    # 콘텐츠 존재성 간단 체크: 본문 텍스트가 너무 적으면 동적일 가능성
    body_text_len = len(soup.get_text(strip=True))
    if body_text_len < 400 and USE_SELENIUM_FALLBACK:
        # 동적 로딩 의심 → Selenium 백업
        html = load_with_selenium(url)
        soup = BeautifulSoup(html, "html.parser")

    sections = guess_sections(soup)

    all_rows: List[ContentRow] = []
    for i, sec in enumerate(sections, start=1):
        sec_title = sec["title"]
        sec_rows = parse_section(sec_title, sec["nodes"])
        # 섹션 순서 태깅
        for r in sec_rows:
            r.section_order = i
        all_rows.extend(sec_rows)
        time.sleep(SLEEP_SEC)

    return all_rows


def save_as_csv(rows: List[ContentRow], path: str):
    with open(path, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["section_order", "section_title", "content_type", "text", "href"])
        for r in rows:
            writer.writerow([r.section_order, r.section_title, r.content_type, r.text, r.href or ""])


def save_as_json(rows: List[ContentRow], path: str):
    with open(path, "w", encoding="utf-8") as f:
        json.dump([asdict(r) for r in rows], f, ensure_ascii=False, indent=2)


In [6]:
safe_mkdir(OUT_DIR)
rows = crawl_cela_isms(TARGET_URL)

# 간단 품질 점검 로그
print(f"[INFO] Extracted rows: {len(rows)}")
cnt_table = sum(1 for r in rows if r.content_type == "table")
cnt_para  = sum(1 for r in rows if r.content_type == "paragraph")
cnt_list  = sum(1 for r in rows if r.content_type == "list")
cnt_link  = sum(1 for r in rows if r.content_type == "link")
print(f"[INFO] table={cnt_table}, paragraph={cnt_para}, list={cnt_list}, link={cnt_link}")

csv_path = os.path.join(OUT_DIR, "cela_isms_1_1.csv")
json_path = os.path.join(OUT_DIR, "cela_isms_1_1.json")
save_as_csv(rows, csv_path)
save_as_json(rows, json_path)
print(f"[OK] Saved: {csv_path}")
print(f"[OK] Saved: {json_path}")

[INFO] Extracted rows: 292
[INFO] table=0, paragraph=195, list=25, link=72
[OK] Saved: ../data/cela_isms_1_1.csv
[OK] Saved: ../data/cela_isms_1_1.json


In [7]:
rows

[ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='정보보호 관리체계 (ISMS)', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='1.1.1. 경 영진의 참여', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='최고경영자는 정보보호 및 개인정보보호 관리체계의 수립과 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 체계를 수립하여 운영하여야 한다.', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='1. 정보보호 및 개인정보보호 관리체계의 수립 및 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 등의 책임과 역할을 문서화하고 있는가? (간편인증-7의2) (간편인증-7의3)', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='- 정보보호 및 개인정보보호 정책의 제· 개정, 위험관리, 내부감사 등 관리체계 운영의 중요 사안에 대하여 경영진이 참여할 수 있도록 활동의 근거를 정보보호 및 개인정보보호 정책 또는 시행문서에 명시', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='2. 경영진이 정보보호 및 개인정보보호 활동에 관한 의사결정에 적극적으로 참여할 수 있는 보고, 검토 및 승인 절차를 수립· 이행하고 있는가? (간편인증-7의2) (간편인증-7의

In [8]:
# -*- coding: utf-8 -*-
"""
CE.LA ISMS 페이지(https://www.cela.kr/ISMS_1_1) 크롤러
- 정적: requests + BeautifulSoup
- 동적: Selenium 백업 + '접기/펼치기/상세' 등 모든 토글 열기
- 표/문단/목록/링크를 구조화하여 CSV/JSON 저장
"""

import os
import time
import json
import csv
from dataclasses import dataclass, asdict
from typing import List, Dict, Optional

import requests
from bs4 import BeautifulSoup

# ===== 설정 =====
TARGET_URL = "https://www.cela.kr/ISMS_1_1"
OUT_DIR = "../data"
TIMEOUT = 20
RETRY = 3
SLEEP_SEC = 0.8
USE_SELENIUM_FALLBACK = True   # 정적 파싱이 비면 Selenium로 재시도(+토글 전부 펼치기)

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/134.0.0.0 Safari/537.36"
    )
}

@dataclass
class ContentRow:
    section_order: int
    section_title: str
    content_type: str   # "paragraph" | "list" | "table" | "link"
    text: str           # 표는 TSV 문자열
    href: Optional[str] = None


def safe_mkdir(path: str):
    os.makedirs(path, exist_ok=True)


def fetch_html(url: str, headers: Dict[str, str], timeout: int = TIMEOUT) -> str:
    last_exc = None
    for _ in range(RETRY):
        try:
            resp = requests.get(url, headers=headers, timeout=timeout)
            if 200 <= resp.status_code < 300:
                return resp.text
        except Exception as e:
            last_exc = e
        time.sleep(0.8)
    if last_exc:
        raise last_exc
    raise RuntimeError("Failed to fetch HTML")


def load_with_selenium_and_expand(url: str) -> str:
    """
    Selenium로 페이지 로드 후, 접기/펼치기/상세 등 숨겨진 영역을 모두 펼친 뒤 HTML 반환.
    - pip install selenium webdriver-manager
    - 로컬/서버에 Chrome/Chromium 필요
    """
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import (
        StaleElementReferenceException,
        ElementClickInterceptedException,
        ElementNotInteractableException,
        TimeoutException,
        WebDriverException,
    )

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--lang=ko-KR")
    options.add_argument(f"--user-agent={HEADERS['User-Agent']}")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=options)
    wait = WebDriverWait(driver, 15)
    try:
        driver.get(url)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        time.sleep(1.2)

        def click_safe(el):
            try:
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
                time.sleep(0.15)
                try:
                    el.click()
                except (ElementClickInterceptedException, ElementNotInteractableException):
                    driver.execute_script("arguments[0].click();", el)
                time.sleep(0.2)
                return True
            except WebDriverException:
                return False

        def expand_round() -> int:
            """한 라운드에서 열린 요소 수(클릭 성공 수) 반환"""
            opened = 0

            # 1) details/summary
            summaries = driver.find_elements(By.CSS_SELECTOR, "details:not([open]) > summary")
            for s in summaries:
                if click_safe(s):
                    opened += 1

            # 2) aria-expanded=false → true로 전환될 토글
            toggles = driver.find_elements(By.CSS_SELECTOR, "[aria-expanded='false'], [aria-pressed='false']")
            for t in toggles:
                if click_safe(t):
                    opened += 1

            # 3) '펼치기/접기/상세/자세히/더보기' 텍스트가 있는 버튼/링크/요소
            keywords = ("펼치기", "접기", "상세", "자세히", "더보기")
            candidates = driver.find_elements(By.CSS_SELECTOR, "button, a, [role='button'], .toggle, .accordion, .collapse, .expander")
            for c in candidates:
                try:
                    txt = (c.text or "").strip()
                except StaleElementReferenceException:
                    continue
                if any(k in txt for k in keywords):
                    if click_safe(c):
                        opened += 1

            # 4) 흔한 아코디언 트리거 클래스 추가 탐색
            more_candidates = driver.find_elements(By.XPATH,
                "//*[contains(@class,'accordion') or contains(@class,'toggle') or contains(@class,'collapse') or contains(@class,'expand')]"
            )
            for m in more_candidates:
                try:
                    txt2 = (m.text or "").strip()
                except StaleElementReferenceException:
                    continue
                if any(k in txt2 for k in keywords):
                    if click_safe(m):
                        opened += 1

            return opened

        # 열릴 게 없을 때까지 반복 (상한선으로 안전장치)
        MAX_ROUNDS = 8
        for _ in range(MAX_ROUNDS):
            changed = expand_round()
            if changed == 0:
                break
            time.sleep(0.6)

        # 모든 콘텐츠가 DOM에 노출되었을 가능성이 높으니 페이지 소스 추출
        time.sleep(0.6)
        html = driver.page_source
        return html

    finally:
        driver.quit()


# ===== 파서 =====
def normalize_text(el) -> str:
    return " ".join(el.get_text(" ", strip=True).split())


def extract_tables(soup: BeautifulSoup) -> List[str]:
    tables_tsv: List[str] = []
    for table in soup.select("table"):
        rows = []
        for tr in table.select("tr"):
            cells = tr.select("th, td")
            row = []
            for c in cells:
                txt = " ".join(c.get_text(" ", strip=True).split())
                row.append(txt)
            rows.append("\t".join(row))
        if rows:
            tables_tsv.append("\n".join(rows))
    return tables_tsv


def guess_sections(soup: BeautifulSoup) -> List[Dict]:
    headers = soup.select("h1, h2, h3, h4")
    sections = []
    for h in headers:
        title = normalize_text(h)
        content_nodes = []
        for sib in h.find_all_next():
            if sib == h:
                continue
            if sib.name in ["h1", "h2", "h3", "h4"]:
                break
            if sib.name in ["script", "style", "noscript"]:
                continue
            content_nodes.append(sib)
        sections.append({"title": title, "nodes": content_nodes})
    if not sections:
        body = soup.body or soup
        sections = [{"title": "본문", "nodes": list(body.children)}]
    return sections


def parse_section(title: str, nodes: List) -> List[ContentRow]:
    rows: List[ContentRow] = []
    order = 0

    temp_soup = BeautifulSoup("".join(str(n) for n in nodes), "html.parser")

    # 표
    for tsv in extract_tables(temp_soup):
        rows.append(ContentRow(order, title, "table", tsv))
        order += 1

    # 문단
    for p in temp_soup.select("p"):
        txt = normalize_text(p)
        if txt:
            rows.append(ContentRow(order, title, "paragraph", txt))
            order += 1

    # 목록 (리스트 내부 모든 항목 수집: 중첩 포함)
    for lst in temp_soup.select("ul, ol"):
        items = []
        for li in lst.select("li"):   # ← 여기! 모든 li 수집
            t = normalize_text(li)
            if t:
                items.append(t)
        if items:
            rows.append(ContentRow(order, title, "list", "\n".join(items)))
            order += 1

    # 링크
    for a in temp_soup.select("a[href]"):
        href = a.get("href", "").strip()
        txt = normalize_text(a)
        if href and not href.startswith("javascript:"):
            rows.append(ContentRow(order, title, "link", txt, href=href))
            order += 1

    return rows


def crawl_cela_isms(url: str) -> List[ContentRow]:
    html = fetch_html(url, HEADERS)
    soup = BeautifulSoup(html, "html.parser")

    # 본문 텍스트가 너무 적으면 동적일 가능성 → Selenium로 전체 펼치기
    if len(soup.get_text(strip=True)) < 600 and USE_SELENIUM_FALLBACK:
        html = load_with_selenium_and_expand(url)
        soup = BeautifulSoup(html, "html.parser")

    sections = guess_sections(soup)

    all_rows: List[ContentRow] = []
    for i, sec in enumerate(sections, start=1):
        sec_title = sec["title"]
        sec_rows = parse_section(sec_title, sec["nodes"])
        for r in sec_rows:
            r.section_order = i
        all_rows.extend(sec_rows)
        time.sleep(SLEEP_SEC)

    return all_rows


def save_as_csv(rows: List[ContentRow], path: str):
    with open(path, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["section_order", "section_title", "content_type", "text", "href"])
        for r in rows:
            writer.writerow([r.section_order, r.section_title, r.content_type, r.text, r.href or ""])


def save_as_json(rows: List[ContentRow], path: str):
    with open(path, "w", encoding="utf-8") as f:
        json.dump([asdict(r) for r in rows], f, ensure_ascii=False, indent=2)


[INFO] Extracted rows: 292
[INFO] table=0, paragraph=195, list=25, link=72
[OK] Saved: ../data/cela_isms_1_1.csv
[OK] Saved: ../data/cela_isms_1_1.json


In [ ]:
safe_mkdir(OUT_DIR)
rows = crawl_cela_isms(TARGET_URL)
print(f"[INFO] Extracted rows: {len(rows)}")
print(f"[INFO] table={sum(r.content_type=='table' for r in rows)}, "
      f"paragraph={sum(r.content_type=='paragraph' for r in rows)}, "
      f"list={sum(r.content_type=='list' for r in rows)}, "
      f"link={sum(r.content_type=='link' for r in rows)}")

csv_path = os.path.join(OUT_DIR, "cela_isms_1_1.csv")
json_path = os.path.join(OUT_DIR, "cela_isms_1_1.json")
save_as_csv(rows, csv_path)
save_as_json(rows, json_path)
print(f"[OK] Saved: {csv_path}")
print(f"[OK] Saved: {json_path}")

In [9]:
rows

[ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='정보보호 관리체계 (ISMS)', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='1.1.1. 경 영진의 참여', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='최고경영자는 정보보호 및 개인정보보호 관리체계의 수립과 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 체계를 수립하여 운영하여야 한다.', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='1. 정보보호 및 개인정보보호 관리체계의 수립 및 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 등의 책임과 역할을 문서화하고 있는가? (간편인증-7의2) (간편인증-7의3)', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='- 정보보호 및 개인정보보호 정책의 제· 개정, 위험관리, 내부감사 등 관리체계 운영의 중요 사안에 대하여 경영진이 참여할 수 있도록 활동의 근거를 정보보호 및 개인정보보호 정책 또는 시행문서에 명시', href=None),
 ContentRow(section_order=1, section_title='본문', content_type='paragraph', text='2. 경영진이 정보보호 및 개인정보보호 활동에 관한 의사결정에 적극적으로 참여할 수 있는 보고, 검토 및 승인 절차를 수립· 이행하고 있는가? (간편인증-7의2) (간편인증-7의

In [20]:
import re
from typing import List, Dict

# === 데이터 클래스 정의 ===
class ContentRow:
    def __init__(self, section_order, section_title, content_type, text, href=None):
        self.section_order = section_order
        self.section_title = section_title
        self.content_type = content_type
        self.text = text
        self.href = href

class LawDoc:
    def __init__(self, text: str, meta: Dict):
        self.text = text
        self.meta = meta

    def __repr__(self):
        return f"LawDoc(text={self.text[:30]}..., meta={self.meta})"


# === 변환 함수 ===
def convert_contentrows_to_lawdocs(rows: List[ContentRow],
                                   law_id="isms",
                                   law_name="ISMS-P",
                                   source_uri="../data/isms.pdf") -> List[LawDoc]:
    lawdocs = []
    buffer_text = []
    article_id, article_title = None, None

    # 정규식: "1.1.1. 경영진의 참여" → 제1.1.1조 (경영진의 참여)
    article_pattern = re.compile(r"^(\d+(\.\d+)*)(?:\.\s*|\s+)(.+)$")

    for row in rows:
        line = row.text.strip()
        if not line:
            continue

        match = article_pattern.match(line)
        if match:
            # 이전 article 저장
            if buffer_text and article_id:
                text = f"{law_name} {article_id}({article_title})\n" + "\n".join(buffer_text)
                meta = {
                    "law_id": law_id,
                    "law_name": law_name,
                    "article_id": article_id,
                    "article_title": article_title,
                    "effective_date": None,
                    "tok_len": len(text.split()),
                    "source_uri": source_uri,
                    "version": None
                }
                lawdocs.append(LawDoc(text, meta))

            # 새 article 시작
            article_id = f"제{match.group(1)}조"
            article_title = match.group(3).strip()
            buffer_text = []
        else:
            buffer_text.append(line)

    # 마지막 article 저장
    if buffer_text and article_id:
        text = f"{law_name} {article_id}({article_title})\n" + "\n".join(buffer_text)
        meta = {
            "law_id": law_id,
            "law_name": law_name,
            "article_id": article_id,
            "article_title": article_title,
            "effective_date": None,
            "tok_len": len(text.split()),
            "source_uri": source_uri,
            "version": None
        }
        lawdocs.append(LawDoc(text, meta))

    return lawdocs


# === 예시 실행 ===

lawdocs = convert_contentrows_to_lawdocs(rows)
for doc in lawdocs:
    print(doc.text)
    print(doc.meta)
    print("####################################################################################################################################################################################################################################")


ISMS-P 제1.1.1조(경 영진의 참여)
최고경영자는 정보보호 및 개인정보보호 관리체계의 수립과 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 체계를 수립하여 운영하여야 한다.
{'law_id': 'isms', 'law_name': 'ISMS-P', 'article_id': '제1.1.1조', 'article_title': '경 영진의 참여', 'effective_date': None, 'tok_len': 24, 'source_uri': '../data/isms.pdf', 'version': None}
####################################################################################################################################################################################################################################
ISMS-P 제1조(정보보호 및 개인정보보호 관리체계의 수립 및 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 등의 책임과 역할을 문서화하고 있는가? (간편인증-7의2) (간편인증-7의3))
- 정보보호 및 개인정보보호 정책의 제· 개정, 위험관리, 내부감사 등 관리체계 운영의 중요 사안에 대하여 경영진이 참여할 수 있도록 활동의 근거를 정보보호 및 개인정보보호 정책 또는 시행문서에 명시
{'law_id': 'isms', 'law_name': 'ISMS-P', 'article_id': '제1조', 'article_title': '정보보호 및 개인정보보호 관리체계의 수립 및 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 등의 책임과 역할을 문서화하고 있는가? (간편인증-7의2) (간편인증-7의3)', 'effective_date': None, 'tok_len': 52, 'source_uri

In [24]:
lawdocs[0].text

'ISMS-P 제1.1.1조(경 영진의 참여)\n최고경영자는 정보보호 및 개인정보보호 관리체계의 수립과 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 체계를 수립하여 운영하여야 한다.'

In [21]:
lawdocs

[LawDoc(text=ISMS-P 제1.1.1조(경 영진의 참여)
 최고경영자..., meta={'law_id': 'isms', 'law_name': 'ISMS-P', 'article_id': '제1.1.1조', 'article_title': '경 영진의 참여', 'effective_date': None, 'tok_len': 24, 'source_uri': '../data/isms.pdf', 'version': None}),
 LawDoc(text=ISMS-P 제1조(정보보호 및 개인정보보호 관리체계의..., meta={'law_id': 'isms', 'law_name': 'ISMS-P', 'article_id': '제1조', 'article_title': '정보보호 및 개인정보보호 관리체계의 수립 및 운영활동 전반에 경영진의 참여가 이루어질 수 있도록 보고 및 의사결정 등의 책임과 역할을 문서화하고 있는가? (간편인증-7의2) (간편인증-7의3)', 'effective_date': None, 'tok_len': 52, 'source_uri': '../data/isms.pdf', 'version': None}),
 LawDoc(text=ISMS-P 제2조(경영진이 정보보호 및 개인정보보호 ..., meta={'law_id': 'isms', 'law_name': 'ISMS-P', 'article_id': '제2조', 'article_title': '경영진이 정보보호 및 개인정보보호 활동에 관한 의사결정에 적극적으로 참여할 수 있는 보고, 검토 및 승인 절차를 수립· 이행하고 있는가? (간편인증-7의2) (간편인증-7의3)', 'effective_date': None, 'tok_len': 88, 'source_uri': '../data/isms.pdf', 'version': None}),
 LawDoc(text=ISMS-P 제1.1.2조(최고책임자의 지정)
 최고경영..., meta={'law_id': 'isms', 'law_name': 'ISMS-P', 'ar

In [23]:
print("""LawDoc(text='개인정보 보호법 제1조(목적)\n이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인 의 존엄과 가치를 구현함을 목적으로 한다.', meta={'law_id': 'pipa', 'law_name': '개인정보 보호법', 'article_id': '제1조', 'article_title': '목적', 'effective_date': None, 'tok_len': 56, 'source_uri': '../data/개인정보 보호법(법률)(제19234호)(20250313).pdf', 'version': None}),
 LawDoc(text='개인정보 보호법 제2조(정의)\n이 법에서 사용하는 용어의 뜻은 다음과 같다. 1. “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 정보를 말한다. 가. 성명, 주민등록번호 및 영상 등을 통하여 개인을 알아볼 수 있는 정보 나. 해당 정보만으로는 특정 개인을 알아볼 수 없더라도 다른 정보와 쉽게 결합하여 알아볼 수 있는 정보. 이 경우 쉽게 결합할 수 있는지 여부는 다른 정보의 입수 가능성 등 개인을 알아보는 데 소요되는 시간, 비용, 기술 등 을 합리적으로 고려하여야 한다. 다. 가목 또는 나목을 제1호의2에 따라 가명처리함으로써 원래의 상태로 복원하기 위한 추가 정보의 사용ㆍ결합 없이는 특정 개인을 알아볼 수 없는 정보(이하 “가명정보”라 한다) 1의2. “가명처리”란 개인정보의 일부를 삭제하거나 일부 또는 전부를 대체하는 등의 방법으로 추가 정보가 없이는 특정 개인을 알아볼 수 없도록 처리하는 것을 말한다. 2. “처리”란 개인정보의 수집, 생성, 연계, 연동, 기록, 저장, 보유, 가공, 편집, 검색, 출력, 정정, 복구, 이용, 제공, 공개, 파기, 그 밖에 이와 유사한 행위를 말한다. 3. “정보주체”란 처리되는 정보에 의하여 알아볼 수 있는 사람으로서 그 정보의 주체가 되는 사람을 말한다. 4. “개인정보파일”이란 개인정보를 쉽게 검색할 수 있도록 일정한 규칙에 따라 체계적으로 배열하거나 구성한 개인 정보의 집합물을 말한다. 5. “개인정보처리자”란 업무를 목적으로 개인정보파일을 운용하기 위하여 스스로 또는 다른 사람을 통하여 개인정 보를 처리하는 공공기관, 법인, 단체 및 개인 등을 말한다. 6. “공공기관”이란 다음 각 목의 기관을 말한다. 가. 국회, 법원, 헌법재판소, 중앙선거관리위원회의 행정사무를 처리하는 기관, 중앙행정기관(대통령 소속 기관과 국무총리 소속 기관을 포함한다) 및 그 소속 기관, 지방자치단체 나. 그 밖의 국가기관 및 공공단체 중 대통령령으로 정하는 기관 7.', meta={'law_id': 'pipa', 'law_name': '개인정보 보호법', 'article_id': '제2조', 'article_title': '정의', 'effective_date': None, 'tok_len': 568, 'source_uri': '../data/개인정보 보호법(법률)(제19234호)(20250313).pdf', 'version': None}),
 LawDoc(text='개인정보 보호법 제2조(정의)\n그 밖의 국가기관 및 공공단체 중 대통령령으로 정하는 기관 7. “고정형 영상정보처리기기”란 일정한 공간에 설치되어 지속적 또는 주기적으로 사람 또는 사물의 영상 등을 촬영 하거나 이를 유ㆍ무선망을 통하여 전송하는 장치로서 대통령령으로 정하는 장치를 말한다. 7의2. “이동형 영상정보처리기기”란 사람이 신체에 착용 또는 휴대하거나 이동 가능한 물체에 부착 또는 거치하여 사람 또는 사물의 영상 등을 촬영하거나 이를 유ㆍ무선망을 통하여 전송하는 장치로서 대통령령으로 정하 는 장치를 말한다. 8. “과학적 연구”란 기술의 개발과 실증, 기초연구, 응용연구 및 민간 투자 연구 등 과학적 방법을 적용하는 연구를 말한다.', meta={'law_id': 'pipa', 'law_name': '개인정보 보호법', 'article_id': '제2조', 'article_title': '정의', 'effective_date': None, 'tok_len': 215, 'source_uri': '../data/개인정보 보호법(법률)(제19234호)(20250313).pdf', 'version': None}),
 LawDoc(text='개인정보 보호법 제3조(개인정보 보호 원칙)\n(1) 개인정보처리자는 개인정보의 처리 목적을 명확하게 하여야 하고 그 목적에 필요한 범위 에서 최소한의 개인정보만을 적법하고 정당하게 수집하여야 한다. (2) 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 적합하게 개인정보를 처리하여야 하며, 그 목적 외 의 용도로 활용하여서는 아니 된다. (3) 개인정보처리자는 개인정보의 처리 목적에 필요한 범위에서 개인정보의 정확성, 완전성 및 최신성이 보장되도록 하여야 한다. (4) 개인정보처리자는 개인정보의 처리 방법 및 종류 등에 따라 정보주체의 권리가 침해받을 가능성과 그 위험 정도 를 고려하여 개인정보를 안전하게 관리하여야 한다. (5) 개인정보처리자는 제30조에 따른 개인정보 처리방침 등 개인정보의 처리에 관한 사항을 공개하여야 하며, 열람 청구권 등 정보주체의 권리를 보장하여야 한다. (6) 개인정보처리자는 정보주체의 사생활 침해를 최소화하는 방법으로 개인정보를 처리하여야 한다. (7) 개인정보처리자는 개인정보를 익명 또는 가명으로 처리하여도 개인정보 수집목적을 달성할 수 있는 경우 익명 처리가 가능한 경우에는 익명에 의하여, 익명처리로 목적을 달성할 수 없는 경우에는 가명에 의하여 처리될 수 있도 록 하여야 한다. (8) 개인정보처리자는 이 법 및 관계 법령에서 규정하고 있는 책임과 의무를 준수하고 실천함으로써 정보주체의 신 뢰를 얻기 위하여 노력하여야 한다.', meta={'law_id': 'pipa', 'law_name': '개인정보 보호법', 'article_id': '제3조', 'article_title': '개인정보 보호 원칙', 'effective_date': None, 'tok_len': 410, 'source_uri': '../data/개인정보 보호법(법률)(제19234호)(20250313).pdf', 'version': None}),
 LawDoc(text='개인정보 보호법 제4조(정보주체의 권리)\n정보주체는 자신의 개인정보 처리와 관련하여 다음 각 호의 권리를 가진다. 1. 개인정보의 처리에 관한 정보를 제공받을 권리 2. 개인정보의 처리에 관한 동의 여부, 동의 범위 등을 선택하고 결정할 권리 3. 개인정보의 처리 여부를 확인하고 개인정보에 대한 열람(사본의 발급을 포함한다. 이하 같다) 및 전송을 요구할 권리 4. 개인정보의 처리 정지, 정정ㆍ삭제 및 파기를 요구할 권리 5. 개인정보의 처리로 인하여 발생한 피해를 신속하고 공정한 절차에 따라 구제받을 권리 6. 완전히 자동화된 개인정보 처리에 따른 결정을 거부하거나 그에 대한 설명 등을 요구할 권리', meta={'law_id': 'pipa', 'law_name': '개인정보 보호법', 'article_id': '제4조', 'article_title': '정보주체의 권리', 'effective_date': None, 'tok_len': 197, 'source_uri': '../data/개인정보 보호법(법률)(제19234호)(20250313).pdf', 'version': None})]""")

LawDoc(text='개인정보 보호법 제1조(목적)
이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인 의 존엄과 가치를 구현함을 목적으로 한다.', meta={'law_id': 'pipa', 'law_name': '개인정보 보호법', 'article_id': '제1조', 'article_title': '목적', 'effective_date': None, 'tok_len': 56, 'source_uri': '../data/개인정보 보호법(법률)(제19234호)(20250313).pdf', 'version': None}),
 LawDoc(text='개인정보 보호법 제2조(정의)
이 법에서 사용하는 용어의 뜻은 다음과 같다. 1. “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 정보를 말한다. 가. 성명, 주민등록번호 및 영상 등을 통하여 개인을 알아볼 수 있는 정보 나. 해당 정보만으로는 특정 개인을 알아볼 수 없더라도 다른 정보와 쉽게 결합하여 알아볼 수 있는 정보. 이 경우 쉽게 결합할 수 있는지 여부는 다른 정보의 입수 가능성 등 개인을 알아보는 데 소요되는 시간, 비용, 기술 등 을 합리적으로 고려하여야 한다. 다. 가목 또는 나목을 제1호의2에 따라 가명처리함으로써 원래의 상태로 복원하기 위한 추가 정보의 사용ㆍ결합 없이는 특정 개인을 알아볼 수 없는 정보(이하 “가명정보”라 한다) 1의2. “가명처리”란 개인정보의 일부를 삭제하거나 일부 또는 전부를 대체하는 등의 방법으로 추가 정보가 없이는 특정 개인을 알아볼 수 없도록 처리하는 것을 말한다. 2. “처리”란 개인정보의 수집, 생성, 연계, 연동, 기록, 저장, 보유, 가공, 편집, 검색, 출력, 정정, 복구, 이용, 제공, 공개, 파기, 그 밖에 이와 유사한 행위를 말한다. 3. “정보주체”란 처리되는 정보에 의하여 알아볼 수 있는 사람으로서 그 정보의 주체가 되는 사람을 말한다. 4. “개인정보파일”이란 개인정